In [10]:
! pip install langchain langchain-community langchain-huggingface transformers accelerate bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 13.0 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of tenacity to determine which version is compatible with other requirements. This could take a while.
  Using cached tenacity-8.3.0-py3-none-any.whl (25 kB)
INFO: pip is looking at multiple versions of sqlalchemy to determine which version is compatible with other requirements. This could take a while.
  Using cached SQLAlchemy-2.0.30-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.1 MB)
INFO: pip is looking at multiple versions of sentence-transformers to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.3/163.3 kB 26.8 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of safetensors to determine which version is compatible with other requirements. This could take a while.
  Using cached safetensors-0.4.3-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86

In [1]:
# To get environment variables
import os
import json

# from rich.pretty import pprint


# To split our transcript into pieces
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Our chat model. We'll use the default which is gpt-3.5-turbo
from langchain.chat_models import ChatOpenAI
# from langchain_community.llms import HuggingFacePipeline
# from langchain_huggingface.llms import HuggingFacePipeline
from langchain_community.chat_models.huggingface import ChatHuggingFace
from langchain.chains.summarize import load_summarize_chain

# Prompt templates for dynamic values
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate, # I included this one so you know you'll have it but we won't be using it
    HumanMessagePromptTemplate
)

from langchain_core.output_parsers import StrOutputParser, JsonOutputParser
from langchain.output_parsers import PydanticOutputParser

# To create our chat messages
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)

from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_core.prompt_values import StringPromptValue
from typing import List, Optional

from langchain.llms.huggingface_pipeline import HuggingFacePipeline
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from transformers import BitsAndBytesConfig

/home/jon/Code/hackathon/til-24-base/nlp/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
nf4_config = BitsAndBytesConfig(
   load_in_4bit=True,
   bnb_4bit_quant_type="nf4",
   bnb_4bit_use_double_quant=True,
   bnb_4bit_compute_dtype=torch.bfloat16
)

model_id = "microsoft/Phi-3-mini-4k-instruct"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
                 model_id,
                 quantization_config=nf4_config
                 )

#tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
                 model_id,
                 quantization_config=nf4_config
                 )

# tokenizer.save_pretrained('./src/models/tokenizer')
# model.save_pretrained('./src/models/model')

pipe = pipeline("text-generation",
               model=model,
               tokenizer=tokenizer,
               max_new_tokens=512,
               )

llm_phi3 = HuggingFacePipeline(pipeline=pipe)

# llm_phi3 = HuggingFacePipeline.from_model_id(
#     model_id="microsoft/Phi-3-mini-4k-instruct",
#     task="text-generation",
#     device=0,
#     pipeline_kwargs={"max_new_tokens": 512, "temperature": 0.1,},
# )

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
`low_cpu_mem_usage` was None, now set to True since model is quantized.
Loading checkpoint shards: 100%|██████████| 2/2 [00:03<00:00,  1.90s/it]
`low_cpu_mem_usage` was None, now set to True since model is quantized.
Loading checkpoint shards: 100%|██████████| 2/2 [00:03<00:00,  1.76s/it]
/home/jon/Code/hackathon/til-24-base/nlp/venv/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 0.3. An updated version of the class exists in the from rom langchain-huggingface package and should be used instead. To use it run `pip install -U from rom langchain-huggingface` and import as `from from rom langchain_huggingface import llms import HuggingFacePipeline`.
  warn_deprecated(


In [3]:
template = """
Given the transcript delimitted by triple backticks,
extract all key information in a structured format.
The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"description": "The data to be extracted from the transcript", "properties": {"target": {"title": "Target", "description": "Identified Target object to attack", "type": "string"}, "heading": {"title": "Heading", "description": "Heading of object in integers", "type": "string"}, "tool": {"title": "Tool", "description": "Tool to be deployed to neutralize the target", "type": "string"}}, "required": ["target", "heading", "tool"]}\n```
Return the JSON object only, nothing else.

Here are some examples of how to perform this task:
TRANSCRIPT: Target is red helicopter, heading is zero one zero, tool to deploy is surface-to-air missiles.
OUTPUT: 'target': 'red helicopter', 'heading': '010', 'tool': 'surface-to-air missiles'

TRANSCRIPT: Target is grey and white fighter jet, heading is zero six five, tool to deploy is electromagnetic pulse.
OUTPUT: 'target': 'grey and white fighter jet', 'heading': '065', 'tool': 'electromagnetic pulse'

TRANSCRIPT: Alfa, Echo, Mike Papa, deploy EMP tool heading zero eight five, engage purple, red, and silver fighter jet.
OUTPUT: 'target': 'purple, red, and silver fighter jet', 'heading': '085', 'tool': 'EMP'

Now take the following transcript and extract the key information in a structured format
TRANSCRIPT:
"""

In [4]:
transcript = "Target is blue stealth bomber, heading is zero five six, tool to deploy is cyber attack"

In [5]:
def clean_json_string(json_str: str) -> str:
    # Find the start of the first JSON object
    start = json_str.find('{')

    # Find the end of the first JSON object
    end = json_str.find('}') + 1

    # Extract the substring from 'start' to 'end'
    cleaned_json_str = json_str[start:end]

    return cleaned_json_str

def extract_json_after_response(message: str) -> dict:
    # Find the start of the JSON object
    start = message.lower().find('response') + len('response')

    # Extract the substring from 'start' to the end of the message
    json_str = message[start:].strip()
    json_str = clean_json_string(json_str)

    try:
        # Parse the JSON string into a Python dictionary
        data = json.loads(json_str)
    except json.JSONDecodeError:
        print(f"Error parsing JSON string: {json_str}")
        # raise
        return None

    return data

In [6]:
response = llm_phi3.predict(f"{template}```{transcript}```")
response

/opt/conda/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `BaseLLM.predict` was deprecated in langchain-core 0.1.7 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(
You are not running the flash-attention implementation, expect numerical differences.


'\nGiven the transcript delimitted by triple backticks,\nextract all key information in a structured format.\nThe output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"description": "The data to be extracted from the transcript", "properties": {"target": {"title": "Target", "description": "Identified Target object to attack", "type": "string"}, "heading": {"title": "Heading", "description": "Heading of object in integers", "type": "string"}, "tool": {"title": "Tool", "description": "Tool to be deployed to neutralize the target", "type": "string"}}, "required": ["target", "heading", "tool"]}\

In [7]:
extract_json_after_response(response)

{'target': 'blue stealth bomber', 'heading': '056', 'tool': 'cyber attack'}

In [8]:
print(f"{template}```{transcript}```")


Given the transcript delimitted by triple backticks,
extract all key information in a structured format.
The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"description": "The data to be extracted from the transcript", "properties": {"target": {"title": "Target", "description": "Identified Target object to attack", "type": "string"}, "heading": {"title": "Heading", "description": "Heading of object in integers", "type": "string"}, "tool": {"title": "Tool", "description": "Tool to be deployed to neutralize the target", "type": "string"}}, "required": ["target", "heading", "tool"]}
```
Return 

In [ ]:
from tqdm import tqdm
import json

def load_test_cases(filename: str) -> list:
  test_cases = []
  with open(filename, 'r') as file:
    for line in file:
      # Each line is a separate JSON object
      test_case = json.loads(line)
      test_cases.append(test_case)
  return test_cases

# Load the test cases from the JSONL file
test_cases = load_test_cases('./src/nlp.jsonl')
print(f"Loaded {len(test_cases)} test cases.")

# Counter for passed test cases
passed_tests = 0

# Now you can use 'test_cases' in your tests
for test_case in tqdm(test_cases):
  transcript = test_case['transcript']
  expected_output = {
    'target': test_case['target'],
    'heading': test_case['heading'],
    'tool': test_case['tool']
  }
  # Check if transcript is an instance of StringPromptValue and extract the string value
  if isinstance(transcript, StringPromptValue):
    print(transcript)
    transcript = transcript.value

  # Invoke the chain with the transcript from the test case
  extracted_output = llm_phi3.predict(f"{template}```{transcript}```")
  extracted_output = extract_json_after_response(extracted_output)

  # Check if the extracted output matches the expected output
  if extracted_output == expected_output:
    passed_tests += 1
  else:
    print(f"For transcript: {transcript}, expected: {expected_output}, but got: {extracted_output}")

# Print the number of passed tests
print(f"Passed {passed_tests} out of {len(test_cases)} tests.")

In [ ]:
#Test

In [ ]:
template = """
Given the transcript delimitted by triple backticks,
extract all key information in a structured format.
{format_instructions}
Return the JSON object only, nothing else.

Here are some examples of how to perform this task:
TRANSCRIPT: Target is red helicopter, heading is zero one zero, tool to deploy is surface-to-air missiles.
OUTPUT: 'target': 'red helicopter', 'heading': '010', 'tool': 'surface-to-air missiles'

TRANSCRIPT: Target is grey and white fighter jet, heading is zero six five, tool to deploy is electromagnetic pulse.
OUTPUT: 'target': 'grey and white fighter jet', 'heading': '065', 'tool': 'electromagnetic pulse'

TRANSCRIPT: Alfa, Echo, Mike Papa, deploy EMP tool heading zero eight five, engage purple, red, and silver fighter jet.
OUTPUT: 'target': 'purple, red, and silver fighter jet', 'heading': '085', 'tool': 'EMP'

Now take the following transcript and extract the key information in a structured format
TRANSCRIPT:
```{transcript}```
"""


# Define your desired data structure.
class Objective(BaseModel):
    """The data to be extracted from the transcript"""
    target: str = Field(description="Identified Target object to attack")
    heading: str = Field(description="Heading of object in integers")
    tool: str = Field(description="Tool to be deployed to neutralize the target")

def clean_json_string(json_str: str) -> str:
    # Find the start of the first JSON object
    start = json_str.find('{')

    # Find the end of the first JSON object
    end = json_str.find('}') + 1

    # Extract the substring from 'start' to 'end'
    cleaned_json_str = json_str[start:end]

    return cleaned_json_str

def extract_json_after_response(message: str) -> dict:
    # Find the start of the JSON object
    start = message.lower().find('response') + len('response')

    # Extract the substring from 'start' to the end of the message
    json_str = message[start:].strip()
    json_str = clean_json_string(json_str)

    try:
        # Parse the JSON string into a Python dictionary
        data = json.loads(json_str)
    except json.JSONDecodeError:
        print(f"Error parsing JSON string: {json_str}")
        return None

    return data

output_parser = JsonOutputParser(pydantic_object=Objective)

prompt = PromptTemplate(
    template=template,
    input_variables=["transcript"],
    partial_variables={"format_instructions": output_parser.get_format_instructions()},
)

chain = prompt | llm_phi3 | extract_json_after_response


from tqdm import tqdm
import json

def load_test_cases(filename: str) -> list:
  test_cases = []
  with open(filename, 'r') as file:
    for line in file:
      # Each line is a separate JSON object
      test_case = json.loads(line)
      test_cases.append(test_case)
  return test_cases

# Load the test cases from the JSONL file
test_cases = load_test_cases('./data/nlp.jsonl')
print(f"Loaded {len(test_cases)} test cases.")

# Counter for passed test cases
passed_tests = 0

# Now you can use 'test_cases' in your tests
for test_case in tqdm(test_cases):
  transcript = test_case['transcript']
  expected_output = {
    'target': test_case['target'],
    'heading': test_case['heading'],
    'tool': test_case['tool']
  }
  # Check if transcript is an instance of StringPromptValue and extract the string value
  if isinstance(transcript, StringPromptValue):
    print(transcript)
    transcript = transcript.value

  # Invoke the chain with the transcript from the test case
  extracted_output = chain.invoke({"transcript": transcript})


  # Check if the extracted output matches the expected output
  if extracted_output == expected_output:
    passed_tests += 1
  else:
    print(f"For transcript: {transcript}, expected: {expected_output}, but got: {extracted_output}")

# Print the number of passed tests
print(f"Passed {passed_tests} out of {len(test_cases)} tests.")

In [7]:
### Test singular test case

template = """
Given the transcript delimitted by triple backticks,
extract all key information in a structured format.
{format_instructions}
Return the JSON object only, nothing else.

Here are some examples of how to perform this task:
TRANSCRIPT: Target is red helicopter, heading is zero one zero, tool to deploy is surface-to-air missiles.
OUTPUT: 'target': 'red helicopter', 'heading': '010', 'tool': 'surface-to-air missiles'

TRANSCRIPT: Target is grey and white fighter jet, heading is zero six five, tool to deploy is electromagnetic pulse.
OUTPUT: 'target': 'grey and white fighter jet', 'heading': '065', 'tool': 'electromagnetic pulse'

TRANSCRIPT: Alfa, Echo, Mike Papa, deploy EMP tool heading zero eight five, engage purple, red, and silver fighter jet.
OUTPUT: 'target': 'purple, red, and silver fighter jet', 'heading': '085', 'tool': 'EMP'

Now take the following transcript and extract the key information in a structured format
TRANSCRIPT:
```{transcript}```
"""


# Define your desired data structure.
class Objective(BaseModel):
    """The data to be extracted from the transcript"""
    target: str = Field(description="Identified Target object to attack")
    heading: str = Field(description="Heading of object in integers")
    tool: str = Field(description="Tool to be deployed to neutralize the target")

def clean_json_string(json_str: str) -> str:
    # Find the start of the first JSON object
    start = json_str.find('{')

    # Find the end of the first JSON object
    end = json_str.find('}') + 1

    # Extract the substring from 'start' to 'end'
    cleaned_json_str = json_str[start:end]

    return cleaned_json_str

def extract_json_after_response(message: str) -> dict:
    # Find the start of the JSON object
    start = message.lower().find('response') + len('response')

    # Extract the substring from 'start' to the end of the message
    json_str = message[start:].strip()
    json_str = clean_json_string(json_str)

    try:
        # Parse the JSON string into a Python dictionary
        data = json.loads(json_str)
    except json.JSONDecodeError:
        print(f"Error parsing JSON string: {json_str}")
        return None

    return data

output_parser = JsonOutputParser(pydantic_object=Objective)

prompt = PromptTemplate(
    template=template,
    input_variables=["transcript"],
    partial_variables={"format_instructions": output_parser.get_format_instructions()},
)

chain = prompt | llm_phi3 | extract_json_after_response


from tqdm import tqdm
import json

def load_test_cases(filename: str) -> list:
  test_cases = []
  with open(filename, 'r') as file:
    for line in file:
      # Each line is a separate JSON object
      test_case = json.loads(line)
      test_cases.append(test_case)
  return test_cases

# Load the test cases from the JSONL file
test_cases = load_test_cases('./data/nlp.jsonl')
print(f"Loaded {len(test_cases)} test cases.")

# Counter for passed test cases
passed_tests = 0

# Get the 269th test case
test_case = test_cases[269]

transcript = test_case['transcript']
expected_output = {
  'target': test_case['target'],
  'heading': test_case['heading'],
  'tool': test_case['tool']
}

# Check if transcript is an instance of StringPromptValue and extract the string value
if isinstance(transcript, StringPromptValue):
  print(transcript)
  transcript = transcript.value

# Invoke the chain with the transcript from the test case
extracted_output = chain.invoke({"transcript": transcript})

# Check if the extracted output matches the expected output
if extracted_output == expected_output:
  passed_tests += 1
  print(f"Test case 269 passed.")
else:
  print(f"For transcript: {transcript}, expected: {expected_output}, but got: {extracted_output}")

# Print the number of passed tests
print(f"Passed {passed_tests} out of 1 test.")

Loaded 3500 test cases.
Test case 269 passed.
Passed 1 out of 1 test.


In [3]:
import pandas as pd
import json

# Load the CSV file
df = pd.read_csv('../results/nlp_results.csv')

# Convert the 'truth' and 'prediction' columns from string to dictionary
df['truth'] = df['truth'].apply(lambda x: json.loads(x.replace("'", '"')))
df['prediction'] = df['prediction'].apply(lambda x: json.loads(x.replace("'", '"')))

# Check how many predictions match the truth
matches = sum(df['truth'] == df['prediction'])
print(f'Number of matches: {matches}')

Number of matches: 568
